In [1]:
import re
from collections import OrderedDict
from pathlib import Path

import pandas as pd

In [2]:
this_dir = Path(".").resolve()

In [3]:
rooms = pd.read_csv(this_dir / "input" / "room_mapping.csv")
hotels = pd.read_csv(this_dir / "input" / "hotel_mapping.csv")

In [10]:
hotel_id_mapping = dict(zip(hotels["external_name"], hotels["hotel_id"]))

In [11]:
rooms["hotel_id"] = rooms["HotelName"].map(hotel_id_mapping)

In [15]:
replacement_mapping = OrderedDict(
    {
        r"Single Use": " ",
        r"Single": " ",
        r"Singe": " ",
        r"\( \)": " ",
        r"Twin Rm/": "Twin Room",
        r"Bedrooms": "Bedroom",
        r"01": "One",
        r"02": "Two",
        r"03": "Three",
        r"1 Bedroom": "One Bedroom",
        r"2 Bedroom": "Two Bedroom",
        r"3 Bedroom": "Three Bedroom",
        r"4 Bedroom": "Four Bedroom",
        r"1-": "One ",
        r"2-": "Two ",
        r"3-": "Three ",
        r"2-Bdrm": "Two Bedroom",
    }
)


def replace_strings(s):
    for pattern, replacement in replacement_mapping.items():
        s = re.sub(pattern, replacement, s, flags=re.IGNORECASE)
    return s.strip()

In [16]:
rooms["updated_room_name"] = rooms["HotelRoomDescription"].apply(replace_strings)
rooms["updated_room_name"] = (
    rooms["updated_room_name"].str.split().str.join(" ").str.title()
)

In [17]:
rooms.drop_duplicates(subset=["hotel_id", "MapRoomCode"], keep="last", inplace=True)

In [18]:
rooms_unique = rooms.groupby(["updated_room_name", "hotel_id"]).size().reset_index()
rooms_unique.sort_values(["hotel_id", "updated_room_name"], inplace=True)

In [19]:
rooms.to_csv("rooms.csv", index=False)
rooms_unique.to_csv("rooms_unique.csv", index=False)

In [20]:
import json

output = []

for _, row in rooms_unique.iterrows():
    hotel_room_dict = {
        "model": "accommodation.hotelroom",
        "fields": {
            "hotel": row["hotel_id"],
            "name": row["updated_room_name"],
        },
    }
    output.append(hotel_room_dict)

output_json = json.dumps(output, indent=2, default=str)

with open("hotelroom.json", "w") as f:
    f.write(output_json)

In [21]:
accommodation_hotel_room = pd.read_csv(this_dir / "input" / "accommodation_hotel_room.csv")

In [22]:
accommodation_hotel_room.head()

,id,name,ordinal,hotel_id
0,1,Artist Village Suite,1,10000072
1,2,Bedouin Room,1,10000072
2,3,Farmstay Suite,1,10000072
3,4,Farmstay Suite King,1,10000072
4,5,Glamping King Room,1,10000072


In [23]:
hotel_room_id_mapping = dict(
    zip(
        accommodation_hotel_room.apply(lambda x: (x["name"], x["hotel_id"]), axis=1),
        accommodation_hotel_room["id"],
    )
)

In [26]:
rooms["room_id"] = rooms.apply(
    lambda x: hotel_room_id_mapping.get(
        (
            x["updated_room_name"],
            x["hotel_id"],
        )
    ),
    axis=1,
)

In [27]:
rooms.columns

Index(['HotelName', 'Destination', 'ContractID', 'OperatorName', 'ContractSPO',
       'Guarantee', 'SingleSupplement', 'SPOName', 'SPOCode', 'contractType',
       'Active', 'BeginDate', 'EndDate', 'ResStartDate', 'ResStopDate',
       'PurchaseContractID', 'Purchase_Room_ID', 'RoomId2', 'RoomDescription',
       'RoomLocation', 'RoomView', 'HotelRoomDescription', 'MapRoomCode',
       'MapRoomCode2', 'FTIRoomCode', 'BIGXRoomCode', 'RoomAllotment',
       'AllotmentGuarantee', 'PriceType', 'PerRoomCalcType', 'MinRoomPrice',
       'MaxOccupancy', 'MinOccupancy', 'MaxAdl1', 'MinAdl', 'MaxChl', 'MinChl',
       'AdultOnly1', 'InfantAllowed1', 'MealPlan', 'hotel_id',
       'updated_room_name', 'room_id'],
      dtype='object')

In [28]:
output = []

for _, row in rooms.iterrows():
    hotel_room_mapping_dict = {
        "model": "mapping.hotelroommapping",
        "fields": {
            "hotel_room": row["room_id"],
            "external_name": row["HotelRoomDescription"],
            "external_code": row["MapRoomCode"],
            "provider": 1,
        },
    }
    output.append(hotel_room_mapping_dict)

output_json = json.dumps(output, indent=2, default=str)

with open("hotelroommapping.json", "w") as f:
    f.write(output_json)